In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

In [2]:
segment = pd.read_csv('segment_info.csv')
warrant = pd.read_csv('warrant_info.csv')
stock_daily = pd.read_csv('Stock_data.csv')

In [3]:
Stock_final = pd.merge(stock_daily, segment[['Stock','Board','Sector']], on='Stock', how='left')
Stock_final = pd.merge(Stock_final, warrant[['Stock','Board']], on='Stock', how='left')
Stock_final['Date'] = pd.to_datetime(Stock_final['Date'], dayfirst=True)

In [4]:
Stock_final['High'] = np.where(Stock_final['High'] == '######', 0, Stock_final['High'])
Stock_final['Low'] = np.where(Stock_final['Low'] == '#####', 0, Stock_final['Low'])
Stock_final['Board_x'] = np.where(Stock_final['Board_y'] == 'STRCWARR', 'STRCWARR', Stock_final['Board_x'])
Stock_final['Sector'] = np.where(Stock_final['Board_y'] == 'STRCWARR', 'STRCWARR', Stock_final['Sector'])
Stock_final['Sector'] = np.where(Stock_final['Board_x'] == 'STRCWARR', 'STRCWARR', Stock_final['Sector'])
Stock_final = Stock_final.iloc[:,:-1]
Stock_final = Stock_final.dropna()

In [5]:
Stock_final['pct'] = Stock_final.sort_values('Date').groupby(['Stock_name']).Closing.pct_change()
Stock_final['Return'] = Stock_final.sort_values('Date').groupby(['Stock_name']).Closing.pct_change()
Stock_final['Return'] = np.where(Stock_final['pct'] > 0, 1 ,Stock_final['Return'])
Stock_final['Return'] = np.where(Stock_final['pct'] <= 0, 0 ,Stock_final['Return'])
Stock_final = Stock_final.rename(columns={'Board_x':'Board'})
Stock_final['Date'] = Stock_final['Date'].dt.strftime('%Y/%m/%d')
Stock_final = Stock_final.dropna()

In [6]:
Stock_final.to_csv('Stock_Data_Final.csv',index=False)  